In [1]:
# set cwd and solver
import os
import cobra
cobra.Configuration().solver = "gurobi"
os.chdir('C:/Users/prins/GitHub/Human1_RPE-PR') 

# load model
from pathlib import Path
from cobra.io import read_sbml_model
mitocore = read_sbml_model(Path().cwd() /  "12918_2017_500_MOESM4_ESM.xml")

Model does not contain SBML fbc package information.


Set parameter Username
Academic license - for non-commercial use only - expires 2024-02-03


SBML package 'layout' not supported by cobrapy, information is not parsed
SBML package 'render' not supported by cobrapy, information is not parsed
Use of CHARGE in the notes element is discouraged, use fbc:charge instead: <Species M_10fthf_c "10-Formyltetrahydrofolate">
Use of FORMULA in the notes element is discouraged, use fbc:chemicalFormula instead: <Species M_10fthf_c "10-Formyltetrahydrofolate">
Use of CHARGE in the notes element is discouraged, use fbc:charge instead: <Species M_10fthf_m "10-Formyltetrahydrofolate">
Use of FORMULA in the notes element is discouraged, use fbc:chemicalFormula instead: <Species M_10fthf_m "10-Formyltetrahydrofolate">
Use of CHARGE in the notes element is discouraged, use fbc:charge instead: <Species M_13dpg_c "3-Phospho-D-glyceroyl phosphate">
Use of FORMULA in the notes element is discouraged, use fbc:chemicalFormula instead: <Species M_13dpg_c "3-Phospho-D-glyceroyl phosphate">
Use of CHARGE in the notes element is discouraged, use fbc:charge in

In [2]:
# load combined RPE_PR models
from pathlib import Path
from cobra.io import read_sbml_model
mitocore = read_sbml_model(Path().cwd() /  "mitocore_12918_2017_500_MOESM4_ESM.xml")
mitocore.objective = 'OF_ATP_MitoCore'

In [3]:
essential_rxns = cobra.flux_analysis.find_essential_reactions(mitocore)
essential_rxns

{<Reaction Biomass_MitoCore at 0x10bc1fa5db0>,
 <Reaction EX_biomass_e at 0x10bc1c891e0>,
 <Reaction OF_ATP_MitoCore at 0x10bc1c95360>}

In [4]:
from cobra.flux_analysis import single_reaction_deletion
rxn_kos = single_reaction_deletion(mitocore, mitocore.reactions[:])
rxn_kos['ids'] = sum([list((rxn)) for rxn in rxn_kos['ids']], [])

In [9]:
from src.get_info import make_rxn_df
import pandas as pd
from cobra.flux_analysis import flux_variability_analysis
fba = mitocore.optimize().to_frame()
fva = flux_variability_analysis(mitocore, loopless=False)
pfba= cobra.flux_analysis.pfba(mitocore).to_frame()
pfba.columns=pfba.columns+'_pFBA'
df = make_rxn_df(mitocore)
df = pd.concat([df,fva,fba,pfba],axis=1)
df = df.merge(rxn_kos, left_on='rxn_ID', right_on='ids')
df

,rxn_ID,cell,lb,ub,sbo,name,subsystem,reaction,met_IDs,GPR,minimum,maximum,fluxes,reduced_costs,fluxes_pFBA,reduced_costs_pFBA,ids,growth,status
0,EX_2hb_e,None,-1000.0,1000.0,SBO:0000627,EX_2hb_e,,2-Hydroxybutanoic acid[External] <=>,2hb_e <=>,,0.000,0.000,0.000,-0.0,0.000,2.0,EX_2hb_e,100.892304,optimal
1,EX_ac_e,None,-1000.0,1000.0,SBO:0000627,EX_ac_e,,Acetate[External] <=>,ac_e <=>,,0.000,0.000,0.000,-0.0,0.000,2.0,EX_ac_e,100.892304,optimal
2,EX_acac_e,None,-1000.0,1000.0,SBO:0000627,EX_acac_e,,Acetoacetate[External] <=>,acac_e <=>,,-0.114,-0.114,-0.114,-0.0,-0.114,2.0,EX_acac_e,100.805015,optimal
3,EX_akg_e,None,-1000.0,1000.0,SBO:0000627,EX_akg_e,,2-Oxoglutarate[External] <=>,akg_e <=>,,0.000,0.000,0.000,-0.0,0.000,2.0,EX_akg_e,100.892304,optimal
4,EX_ala_B_e,None,-1000.0,1000.0,SBO:0000627,EX_ala_B_e,,beta-Alanine[External] <=>,ala_B_e <=>,,0.000,0.000,0.000,-0.0,0.000,2.0,EX_ala_B_e,100.892304,optimal
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
550,COt,None,-1000.0,1000.0,,R_COt CO transporter via diffusion,Boundary Conditions - Misc,CO[Cytosol] <=> CO[External],co_c <=> co_e,,0.000,0.000,0.000,-0.0,0.000,2.0,COt,100.892304,optimal
551,NOt,None,-1000.0,1000.0,,R_NOt NO transport (diffusion),Boundary Conditions - Misc,Nitric oxide[External] <=> Nitric oxide[Cytosol],no_e <=> no_c,,0.000,0.000,0.000,-0.0,0.000,2.0,NOt,100.892304,optimal
552,PCHOLHSTDe,None,-1000.0,1000.0,,R_PCHOLHSTDe Phosphatidylcholine,Boundary Conditions - Lipids,Phosphatidylcholine[Cytosol] <=> Phosphatidylc...,pchol_hs_c <=> pchol_hs_e,,0.000,0.000,0.000,-0.0,0.000,2.0,PCHOLHSTDe,100.892304,optimal
553,PSt3,None,-1000.0,1000.0,,R_PSt3 Phosphatidylserine,Boundary Conditions - Lipids,Phosphatidylserine[External] <=> Phosphatidyls...,ps_hs_e <=> ps_hs_c,,0.000,0.000,0.000,-0.0,0.000,2.0,PSt3,100.892304,optimal


In [12]:
df.to_clipboard()